In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re


def scrape_casbah():
    url = "https://www.casbahmusic.com/venues/the-casbah-san-diego/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    show_blocks = soup.find_all("div", class_="event-info-block")

    results = []

    for show in show_blocks:
        # Artist & ticket link
        title_tag = show.find("p", class_="event-title")
        artist = title_tag.a.get_text(strip=True) if title_tag and title_tag.a else None
        ticket_link = title_tag.a['href'] if title_tag and title_tag.a else None

        # Raw MM/DD date string
        raw_date = show.find("p", class_="event-date").get_text(strip=True) if show.find("p", class_="event-date") else None

        # Supporting acts
        support = show.find("p", class_="supporting-talent")
        supporting_acts = support.get_text(strip=True) if support else None

        # Age restriction
        age_tag = show.find("span", class_="ages")
        age_restriction = age_tag.get_text(strip=True) if age_tag else None

        # Price
        price_tag = show.find("span", class_="price")
        price_raw = price_tag.get_text(strip=True) if price_tag else None

        # Genre
        genre_tag = show.find("p", class_="genre")
        genre = genre_tag.get_text(strip=True) if genre_tag else None

        # Show time
        show_time_tag = show.find("span", class_="see-showtime")
        show_time_str = show_time_tag.get_text(strip=True) if show_time_tag else None

        results.append({
            "venue": "Casbah",
            "date": raw_date,
            "artist": artist,
            "supporting_acts": supporting_acts,
            "ticket_link": ticket_link,
            "price": price_raw,
            "genre": genre,
            "age_restriction": age_restriction,
            "show_time_str": show_time_str
        })

    df = pd.DataFrame(results)

    # Smart year logic: infer year rollover for Jan/Feb shows
    def infer_year(date_str):
        try:
            # Remove weekday if present, e.g., "Mon Jul 14" → "Jul 14"
            cleaned = re.sub(r"^\w{3}\s+", "", date_str.strip())
            cleaned = cleaned.replace("\xa0", " ").strip()  # Fix non-breaking spaces
            month_day = datetime.strptime(cleaned, "%b %d")
            today = datetime.today()
            guessed_date = month_day.replace(year=today.year)
            if guessed_date < today:
                guessed_date = guessed_date.replace(year=today.year + 1)
            return guessed_date
        except Exception as e:
            print(f"Could not parse: {date_str} → {e}")
            return pd.NaT

    df["parsed_date"] = df["date"].apply(infer_year)

    # Parse prices
    df["price_min"] = df["price"].str.extract(r"\$(\d+\.\d{2})").astype(float)
    df["price_max"] = df["price"].str.extract(r"\$\d+\.\d{2}-\$(\d+\.\d{2})")[0].astype(float)

    # Timestamp
    df["scraped_at"] = datetime.now().isoformat()

    return df


In [2]:
# Run and preview
casbah_df = scrape_casbah()
casbah_df.head(10)

,venue,date,artist,supporting_acts,ticket_link,price,genre,age_restriction,show_time_str,parsed_date,price_min,price_max,scraped_at
0,Casbah,Tue Jul 1,"Control Defect, Go Scarlet, Club Majo",,https://wl.seetickets.us/event/control-defect-...,$9.00,Alternative,21+,8:30PM,2025-07-01,9.0,NaN,2025-06-09T21:08:50.948696
1,Casbah,Mon Jul 14,"Keep Flying, Out in Front",,https://wl.seetickets.us/event/keep-flying-out...,$15.00,Rock,21+,8:00PM,2025-07-14,15.0,NaN,2025-06-09T21:08:50.948696
2,Casbah,Sun Nov 16,Adam Melchor,,https://www.ticketweb.com/event/adam-melchor-m...,$25.00-$28.00,Indie,All Ages,8:00PM,2025-11-16,25.0,28.0,2025-06-09T21:08:50.948696
3,Casbah,Wed Sep 24,Moon Hooch,Evan Hatfield,https://www.ticketweb.com/event/moon-hooch-bel...,$22.00-$39.00,Instrumental,21+,8:00PM,2025-09-24,22.0,39.0,2025-06-09T21:08:50.948696
4,Casbah,Tue Jul 8,"Ryan Blue and the Bis, Poor Thing, John Meeks",,https://wl.seetickets.us/event/ryan-blue-and-t...,$10.00,Americana,21+,8:00PM,2025-07-08,10.0,NaN,2025-06-09T21:08:50.948696
5,Casbah,Fri Nov 14,Mating Ritual,,https://wl.seetickets.us/event/mating-ritual/6...,$20.00-$25.00,Alternative,21+,8:30PM,2025-11-14,20.0,25.0,2025-06-09T21:08:50.948696
6,Casbah,Sun Oct 26,Water From Your Eyes,Dutch Interior,https://wl.seetickets.us/event/water-from-your...,$20.00-$25.00,Alternative,21+,8:00PM,2025-10-26,20.0,25.0,2025-06-09T21:08:50.948696
7,Casbah,Thu Oct 16,Pool Kids,"Truth Club, PONY",https://link.dice.fm/poolkids25soda,$15.00-$18.00,Emo,21+,7:30PM,2025-10-16,15.0,18.0,2025-06-09T21:08:50.948696
8,Casbah,Sun Nov 9,Neko Case,John Grant,https://www.ticketweb.com/event/neko-case-fall...,$56.00-$98.00,Indie,21+,8:00PM,2025-11-09,56.0,98.0,2025-06-09T21:08:50.948696
9,Casbah,Wed Sep 10,Cass McCombs,,https://wl.seetickets.us/event/cass-mccombs/65...,$22.00-$25.00,Indie,21+,8:30PM,2025-09-10,22.0,25.0,2025-06-09T21:08:50.948696


In [3]:
from datetime import datetime
import re
import uuid

# Function to infer the correct year for each show date
def infer_year(date_str):
    try:
        # Remove weekday prefix and clean up formatting
        cleaned = re.sub(r"^\w{3}\s+", "", date_str.strip())
        cleaned = cleaned.replace("\xa0", " ").strip()
        month_day = datetime.strptime(cleaned, "%b %d")
        today = datetime.today()
        guessed_date = month_day.replace(year=today.year)
        # If the date already passed this year, assume it's for next year
        if guessed_date < today:
            guessed_date = guessed_date.replace(year=today.year + 1)
        return guessed_date
    except Exception as e:
        print(f"Could not parse: {date_str} → {e}")
        return pd.NaT

# Parse the full datetime from date string
casbah_df["parsed_date"] = casbah_df["date"].apply(infer_year)

# Derive the day of the week
casbah_df["day_of_week"] = casbah_df["parsed_date"].dt.day_name()

# And order day of week
dow_order = {
    "Monday": 1, "Tuesday": 2, "Wednesday": 3,
    "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7
}
casbah_df["day_of_week_order"] = casbah_df["day_of_week"].map(dow_order)


# Flag whether the show is on a weekend
casbah_df["is_weekend"] = casbah_df["day_of_week"].isin(["Friday", "Saturday", "Sunday"])

# Convert the parsed date and show time string into a datetime object
casbah_df["show_time"] = pd.to_datetime(
    casbah_df["parsed_date"].dt.date.astype(str) + " " + casbah_df["show_time_str"],
    errors="coerce"
)

# Flag whether the show is considered a "late" show (after 9 PM)
casbah_df["is_late_show"] = casbah_df["show_time"].dt.hour >= 21

# Generate a unique ID for each event so the data can be anonymized or used relationally
casbah_df["event_id"] = [uuid.uuid4().hex for _ in range(len(casbah_df))]


In [4]:
# -------------------------------
# Add 'genre_group' for high-level classification
# -------------------------------

# The genre column contains very specific or niche categories (e.g., "Hyperpop", "Americana").
# To analyze genre-based patterns more meaningfully—especially in dashboards or summary stats—
# it's helpful to group them into broader buckets like "Rock/Alternative", "Folk/Country", etc.

# Define a mapping of detailed genres to broader groups
genre_mapping = {
    "Alternative": "Rock/Alternative",
    "Americana": "Folk/Country",
    "Country": "Folk/Country",
    "DJ/Dance": "Electronic/Dance",
    "Emo": "Rock/Alternative",
    "Folk": "Folk/Country",
    "Hyperpop": "Electronic/Dance",
    "Indie": "Rock/Alternative",
    "Instrumental": "Experimental/Ambient",
    "New Wave": "Rock/Alternative",
    "Post Punk": "Rock/Alternative",
    "Punk": "Punk",
    "R&B": "Soul/R&B",
    "Rock": "Rock/Alternative",
    "Singer/Songwriter": "Folk/Country",
    "World": "Global/World"
}

# Apply the mapping to create a new column
casbah_df["genre_group"] = casbah_df["genre"].map(genre_mapping)

# Optional: Check if any genres were not mapped (should be none)
unmapped_genres = casbah_df[casbah_df["genre_group"].isna()]["genre"].unique()
if len(unmapped_genres) > 0:
    print("Unmapped genres:", unmapped_genres)


In [5]:
casbah_df.head(10)

,venue,date,artist,supporting_acts,ticket_link,price,genre,age_restriction,show_time_str,parsed_date,price_min,price_max,scraped_at,day_of_week,day_of_week_order,is_weekend,show_time,is_late_show,event_id,genre_group
0,Casbah,Tue Jul 1,"Control Defect, Go Scarlet, Club Majo",,https://wl.seetickets.us/event/control-defect-...,$9.00,Alternative,21+,8:30PM,2025-07-01,9.0,NaN,2025-06-09T21:08:50.948696,Tuesday,2,False,2025-07-01 20:30:00,False,c72a029c9ac242609f49eec8ef0ad28a,Rock/Alternative
1,Casbah,Mon Jul 14,"Keep Flying, Out in Front",,https://wl.seetickets.us/event/keep-flying-out...,$15.00,Rock,21+,8:00PM,2025-07-14,15.0,NaN,2025-06-09T21:08:50.948696,Monday,1,False,2025-07-14 20:00:00,False,9eb2f89f94f94b17965b826e26483287,Rock/Alternative
2,Casbah,Sun Nov 16,Adam Melchor,,https://www.ticketweb.com/event/adam-melchor-m...,$25.00-$28.00,Indie,All Ages,8:00PM,2025-11-16,25.0,28.0,2025-06-09T21:08:50.948696,Sunday,7,True,2025-11-16 20:00:00,False,67eb96c48ce54a4591eb0cf4c0343de0,Rock/Alternative
3,Casbah,Wed Sep 24,Moon Hooch,Evan Hatfield,https://www.ticketweb.com/event/moon-hooch-bel...,$22.00-$39.00,Instrumental,21+,8:00PM,2025-09-24,22.0,39.0,2025-06-09T21:08:50.948696,Wednesday,3,False,2025-09-24 20:00:00,False,a183523d43bf44d7ad36001c389398eb,Experimental/Ambient
4,Casbah,Tue Jul 8,"Ryan Blue and the Bis, Poor Thing, John Meeks",,https://wl.seetickets.us/event/ryan-blue-and-t...,$10.00,Americana,21+,8:00PM,2025-07-08,10.0,NaN,2025-06-09T21:08:50.948696,Tuesday,2,False,2025-07-08 20:00:00,False,9a35dc96e06e43069e7e9e4178a8d4d5,Folk/Country
5,Casbah,Fri Nov 14,Mating Ritual,,https://wl.seetickets.us/event/mating-ritual/6...,$20.00-$25.00,Alternative,21+,8:30PM,2025-11-14,20.0,25.0,2025-06-09T21:08:50.948696,Friday,5,True,2025-11-14 20:30:00,False,f34f1170c6d0474083847b2bad09b937,Rock/Alternative
6,Casbah,Sun Oct 26,Water From Your Eyes,Dutch Interior,https://wl.seetickets.us/event/water-from-your...,$20.00-$25.00,Alternative,21+,8:00PM,2025-10-26,20.0,25.0,2025-06-09T21:08:50.948696,Sunday,7,True,2025-10-26 20:00:00,False,ddb3913be5cf4148842855a78467e5d4,Rock/Alternative
7,Casbah,Thu Oct 16,Pool Kids,"Truth Club, PONY",https://link.dice.fm/poolkids25soda,$15.00-$18.00,Emo,21+,7:30PM,2025-10-16,15.0,18.0,2025-06-09T21:08:50.948696,Thursday,4,False,2025-10-16 19:30:00,False,7e9271199bc84933a1d586cca580c896,Rock/Alternative
8,Casbah,Sun Nov 9,Neko Case,John Grant,https://www.ticketweb.com/event/neko-case-fall...,$56.00-$98.00,Indie,21+,8:00PM,2025-11-09,56.0,98.0,2025-06-09T21:08:50.948696,Sunday,7,True,2025-11-09 20:00:00,False,bd94b38afde34e0783df5109aeb3290a,Rock/Alternative
9,Casbah,Wed Sep 10,Cass McCombs,,https://wl.seetickets.us/event/cass-mccombs/65...,$22.00-$25.00,Indie,21+,8:30PM,2025-09-10,22.0,25.0,2025-06-09T21:08:50.948696,Wednesday,3,False,2025-09-10 20:30:00,False,89d1cad041044452ab3c2ed5d2cc2d4a,Rock/Alternative


In [6]:
# Save to CSV
casbah_df.to_csv("casbah_shows.csv", index=False)

In [9]:
import sqlite3

# Connect to SQLite (creates file if it doesn't exist)
conn = sqlite3.connect("music_industry.db")

# Write casbah_df to a table named 'casbah_web_scrape'
casbah_df.to_sql("casbah_web_scrape", conn, if_exists="replace", index=False)

# Optional: preview some rows
preview = pd.read_sql("SELECT * FROM casbah_web_scrape LIMIT 5", conn)
print(preview)

# Close the connection
conn.close()

    venue        date                                         artist  \
0  Casbah   Tue Jul 1          Control Defect, Go Scarlet, Club Majo   
1  Casbah  Mon Jul 14                      Keep Flying, Out in Front   
2  Casbah  Sun Nov 16                                   Adam Melchor   
3  Casbah  Wed Sep 24                                     Moon Hooch   
4  Casbah   Tue Jul 8  Ryan Blue and the Bis, Poor Thing, John Meeks   

  supporting_acts                                        ticket_link  \
0                  https://wl.seetickets.us/event/control-defect-...   
1                  https://wl.seetickets.us/event/keep-flying-out...   
2                  https://www.ticketweb.com/event/adam-melchor-m...   
3   Evan Hatfield  https://www.ticketweb.com/event/moon-hooch-bel...   
4                  https://wl.seetickets.us/event/ryan-blue-and-t...   

           price         genre age_restriction show_time_str  \
0          $9.00   Alternative             21+        8:30PM   
1     